## Group 4 Project
### Members: Austin Mallie, Cynthia Portales-Loebell, Graham Ward
### Course: ADS507 - Fundamentals of Data Engineering

In [1]:
# Import the necessary libraries
import os
import requests
import mysql.connector
import datetime

In [ ]:
# Set API keys
TICKETMASTER_API_KEY = os.getenv('TICKETMASTER_API_KEY')